## 볼린저 밴드 투자 전략
1. 이동 평균선 생성 : 데이터의 20개의 평균 값
2. 상단 밴드 생성 : 이동 평균선 + (2 * 20개의 데이터의 표준편차)
3. 하단 밴드 생성 : 이동 평균선 - (2 * 20개의 데이터의 표준편차)
4. 가격이 하단 밴드보다 낮은 경우 매수
5. 가격이 상단 밴드보다 높은 경우 매도

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
df = pd.read_csv('../../csv/AAPL.csv', index_col='Date')
df.head(1)

In [ ]:
# 결측치, 무한대 데이터를 제거 
flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)

In [ ]:
df = df.loc[~flag, ['Adj Close']]

In [ ]:
df.iloc[0:20, 0].mean()

In [ ]:
## 이동평균선 컬럼을 생성 값들은 결측치로 대입
df['center'] = np.nan

In [ ]:
df.head(3)

In [ ]:
for i in range(20,len(df)+1):
    mean_data = df.iloc[i-20:i, 0].mean()
    df.iloc[i-1, 1] = mean_data

In [ ]:
df.iloc[18:23,]

In [ ]:
## rolling(n) : n만큼 데이터의 개수를 그룹
df['center2'] = df['Adj Close'].rolling(20).mean()

In [ ]:
df.iloc[18:24]

In [ ]:
# 상단 밴드, 하단 밴드 생성 
# 상단 밴드 = 이동 평균선  + (2 * 20개 데이터의 표준편차)
df['ub'] = \
    df['center'] + (2 * df['Adj Close'].rolling(20).std())
# 하단 밴드 = 이동 평균선 - (2 * 20개의 데이터의 표준편차)
df['lb'] = \
    df['center'] - (2 * df['Adj Close'].rolling(20).std())

In [ ]:
df.iloc[18:23, ]

In [ ]:
# index를 시계열로 변경
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
x = df.tail(100).index
price_y = df.tail(100)['Adj Close']
ub_y = df.tail(100)['ub']
lb_y = df.tail(100)['lb']

plt.figure(figsize=(14, 8))
plt.plot(x, price_y)
plt.plot(x, ub_y)
plt.plot(x, lb_y)
plt.show()

In [ ]:
df[['Adj Close', 'ub', 'lb']].plot()

In [ ]:
# 투자 기간 선택 
start = '2010-01-01'

In [ ]:
# start는 시계열로 변경 
start = datetime.strptime(start, '%Y-%m-%d')

In [ ]:
test_df = df.loc[start:,]

In [ ]:
test_df.tail()

In [ ]:
# 구매 상태를 확인하는 컬럼을 생성 
test_df['trade'] = ""

### 보유 내역 추가 
- 조건식 
    - 상단밴드보다 수정 주가가 높거나 같은 경우 
        - 현재 보유상태라면 
            - 매도 (trade = "")
        - 보유 상태가 아니라면
            - 유지 (trade = "")
    - 상단밴드보다 낮고 하단밴드보다 높은 경우 
        - 현재 보유 상태라면 
            - 유지 (trade = "buy")
        - 보유 상태가 아니라면
            - 유지 (trade = "")
    - 하단 밴드보다 낮거나 같은 경우
        - 현재 보유 상태라면
            - 유지 (trade = "buy")
        - 보유 상태가 아니라면
            - 매수 (trade = "buy")

In [ ]:
for i in test_df.index:
    # i가 의미하는것은? test_df의 index 값
    # print(i)
    # 수정 주가가 상단밴드보다 높거나 같은 경우
    if test_df.loc[i, 'Adj Close'] >=  test_df.loc[i, 'ub']:
        # 현재 보유중이라면 -> trade를 ""로 변경
        # 전날의 trade가 buy라면 -> 매도
        if test_df.shift().loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = ""
        # 보유중이 아니라면
        else:
            test_df.loc[i, 'trade'] = ""
    # 하단밴드보다 수정 주가가 낮거나 같은 경우 
    elif test_df.loc[i, 'Adj Close'] <= test_df.loc[i, 'lb']:
        # 현재 보유중이라면 -> trade를 "buy"로 변경
        if test_df.shift().loc[i, 'trade'] == "buy":
            test_df.loc[i, 'trade'] = "buy"
        # 현재 보유중이 아니라면 -> 매수
        else:
            test_df.loc[i, 'trade'] = "buy"
    # 수정 주가가 밴드 사이에 있을때 
    else:
        # 현재 보유 상태라면 -> 유지 trade를 "buy"
        if test_df.shift().loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = "buy"
        # 보유 상태가 아니라면 -> 유지 trade를 ""
        else:
            test_df.loc[i, 'trade'] = ""

In [ ]:
test_df['trade'].value_counts()

### 수익율 계산
- 구매한 날의 수정 주가 
    - 전날의 trade = "" 오늘의  trade = "buy"인 날의 수정 주가 
- 판매한 날의 수정 주가 
    - 전날의 trade = "buy" 오늘의 trade = ""인 날의 수정 주가 
- 수익율
    - 판매한 날의 수정주가 / 구매한 날의 수정주가 

In [ ]:
# 수익율 파생 변수를 생성 
test_df['rtn'] = 1

for i in test_df.index:
    # 구매가 생성 
    if (test_df.shift().loc[i, 'trade'] == "") & \
        (test_df.loc[i, 'trade'] == "buy"):
        buy = test_df.loc[i, 'Adj Close']
        print(f"매수일 : {i}, 매수가 : {buy}")
    # 판매가 생성
    elif (test_df.shift().loc[i, 'trade'] == "buy") & \
        (test_df.loc[i, 'trade'] == ""):
        sell = test_df.loc[i, 'Adj Close']
        # 수익율 계산
        rtn = sell / buy
        # 계산 된 수익율을 test_df에 rtn 컬럼에 대입
        test_df.loc[i, 'rtn'] = rtn
        print(f"매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}")

In [ ]:
acc_rtn = 1

for i in test_df.index:
    rtn = test_df.loc[i, 'rtn']
    acc_rtn *= rtn

acc_rtn


In [ ]:
test_df['acc_rtn'] = test_df['rtn'].cumprod()

In [ ]:
test_df.iloc[-1,]['acc_rtn']

### 볼린저 밴드의 함수화 
1. 밴드를 생성하는 함수 
    - 매개변수 4개 (데이터프레임, 기준되는 컬럼명, 시작시간, 종료시간, 데이터의 개수)
    - 기준이 되는 컬럼은 기본값을 Adj Close
    - 시작시간은 기본값 2010-01-01 
    - 종료시간은 기본값 ""
    - 데이터의 개수는 기본값은 20
    - 인덱스가 Date인지 확인하여 아니라면 Date를 인덱스로 변경 
    - 인덱스를 시계열 데이터로 변경 
    - 시작시간, 종료시간을 시계열 데이터로 변경 
    - 결측치, 무한대 값을 제외 
    - 기준이되는 컬럼을 제외하고 모두 삭제 
    - 이동 평균선, 상단밴드, 하단밴드 생성
    - 시작시간, 종료시간으로 데이터 필터 
    - 위의 과정에서 나온 데이터프레임을 리턴

In [ ]:
def create_band(
        _df, 
        _col = 'Adj Close', 
        _start = "2010-01-01", 
        _end = datetime.now(), 
        _cnt = 20):
        #복사본 생성
        df = _df.copy()
        # 인덱스가 Date 인가?
        if 'Date' in df.columns:
                df.set_index('Date', inplace=True)

        # index를 시계열 데이터로 변경
        df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
        # 시작시간과 종료 시간은 시계열로 변경 
        try:
                start = datetime.strptime(_start, '%Y-%m-%d')
                if type(_end) == "str":
                        end = datetime.strptime(_end, '%Y-%m-%d')
                else:
                        end = _end
        except:
                return "인자값의 타입이 잘못되었습니다.(예 : YYYY-mm-dd)"
        # 결측치와 무한대 값을 제외
        flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        df = df.loc[~flag,]
        # 기준이 되는 컬럼을 제외하고 모두 삭제 
        result = df[[_col]]
        # 이동 평균선, 상단밴드, 하단밴드 생성
        result['center'] = result[_col].rolling(_cnt).mean()
        result['ub'] = result['center'] + (2 * result[_col].rolling(_cnt).std())
        result['lb'] = result['center'] - (2 * result[_col].rolling(_cnt).std())
        # 시작 시간과 종료시간으로 필터링
        result = result.loc[start:end,]
        return result
        

In [ ]:
df = pd.read_csv("../../csv/AMZN.csv")

In [ ]:
band_df = create_band(df)

In [ ]:
band_df.head()

- 트레이드 컬럼을 생성하는 함수 
    - 매개변수 1개 -> 1번 함수에서 생성된 데이터프레임 인자 값으로 받아온다. 
    - trade 컬럼을 생성하여 값들은 "" 대입 
    - 밴드들을 이용하여 보유 상태를 대입 
    - 결과값을 리턴

In [65]:
def create_trade(_df):
    # 기준이 되는 컬럼의 이름 변수에 저장
    col = _df.columns[0]

    df = _df.copy()

    # 거래 내역이라는 컬럼을 생성
    df['trade'] = ""

    # 거래 내역 추가 
    for i in df.index:
        # 상단밴드보다 기준이 되는 컬럼의 값이 높거나 같은 경우 
        if df.loc[i, col] >= df.loc[i, 'ub']:
            df.loc[i, 'trade'] = ""
        # 하단밴드보다 col의 값이 작거나 같은 경우
        elif df.loc[i, col] <= df.loc[i, 'lb']:
            df.loc[i, 'trade'] = "buy"
        # 밴드 사이에 col의 값이 존재한다면
        else:
            # if df.shift().loc[i, 'trade'] == 'buy':
            #     df.loc[i, 'trade'] = "buy"
            # else:
            #     df.loc[i, 'trade'] = ""
            df.loc[i, 'trade'] = df.shift().loc[i, 'trade']
    return df